In [46]:
from nltk import wsd, pos_tag
from nltk.tokenize import word_tokenize as wt
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
from pprint import pprint
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stopwords

#you'll probably need to download some nltk packages
#it should tell you what they are when you run this cell

ic = wn.ic(brown) 
sw = []
for w in stopwords:
    
    sw.append(w)
    upper = w[0].upper() + w[1:]
    sw.append(upper)
print(sw)

    




['except', 'Except', 'thereby', 'Thereby', 'in', 'In', 'those', 'Those', 'whose', 'Whose', 'your', 'Your', 'take', 'Take', 'much', 'Much', 'five', 'Five', 'yours', 'Yours', 'co', 'Co', 'both', 'Both', 'since', 'Since', 'she', 'She', 'has', 'Has', 'so', 'So', 'four', 'Four', 'side', 'Side', 'call', 'Call', 'forty', 'Forty', 'through', 'Through', 'ie', 'Ie', 'the', 'The', 'nobody', 'Nobody', 'eg', 'Eg', 'are', 'Are', 'another', 'Another', 'cant', 'Cant', 'been', 'Been', 'rather', 'Rather', 'than', 'Than', 'serious', 'Serious', 'seeming', 'Seeming', 'something', 'Something', 'namely', 'Namely', 'interest', 'Interest', 'here', 'Here', 'all', 'All', 'above', 'Above', 'well', 'Well', 'be', 'Be', 'against', 'Against', 'yourself', 'Yourself', 'amongst', 'Amongst', 'by', 'By', 'ourselves', 'Ourselves', 'what', 'What', 'move', 'Move', 'us', 'Us', 'a', 'A', 'up', 'Up', 'former', 'Former', 'now', 'Now', 'anyhow', 'Anyhow', 'system', 'System', 'cry', 'Cry', 'but', 'But', 'am', 'Am', 'thick', 'Thick

In [2]:
#the nltk pos tagger tags differently than wordnet, so I need this helper function
def get_pos(tagged_word):
    

    
    if tagged_word[1][0] == 'N':
        pos = 'n'
    elif tagged_word[1][0] == 'V':
        pos = 'v'
    elif tagged_word[1][0] == 'J':
        pos = 'a'
    elif tagged_word[1][0] == 'R':
        pos = 'r'
    else:
        return None
        
    return pos

In [3]:
#helper to remove stopwords
def filter(tokens):

    filtered = [w for w in tokens if not w in sw and w.isalpha()]
    return filtered

In [4]:
def make_sent_hypernyms(input_sent):
    input_sent_tokens = filter(wt(input_sent))
    sent_hypernyms = {}
    for word in input_sent_tokens:
        if word not in sent_hypernyms.keys():
            sent_hypernyms[word] = get_broader_senses(word)
    
    return sent_hypernyms

In [18]:
def get_broader_senses(word):
    senses = []
    for ss in wn.synsets(word):
        path = []
        result = []
        seen = set()
        todo = [ss]
        while todo:
            next_synset = todo.pop()
            path.append(next_synset)
            if next_synset not in seen:
                seen.add(next_synset)
                next_hypernyms = (
                    next_synset.hypernyms() + next_synset.instance_hypernyms()
                )
                if not next_hypernyms:
                    result.append(next_synset)
                else:
                    todo.extend(next_hypernyms)
        try:
            #print(ss, ss.definition(), path[3], path[3].definition())
            if path[3] not in senses:
                senses.append(path[3])
        except IndexError:
            #print(ss, ss.definition(), path[-1], path[-1].definition())
            if path[-1] not in senses:
                senses.append(path[-1])
        
    return(senses)
    


In [6]:
#provides dictionary with keys of nonstopwords from the input
#which map to their basic wsd lesk-algorithm-provided senses
#also provides lesk algorithm's assessment of our amibguous concept, which rarely goes well for ambiguous concepts
def parse_input(my_word, input_sent):
    my_pos = ''
    input_sent = wt(input_sent)
    filtered_input = filter(input_sent)
    pos_input = pos_tag(input_sent)
    naive_lesk_result = ()
    input_senses = []
    for word in pos_input:

        if word[0] != my_word and word[0] in filtered_input:
            try:
                #lesk_result = wsd.lesk(input_sent, word[0], get_pos(word))
                #input_lesk_senses[word[0]] = (lesk_result, lesk_result.definition())
                senses = [ss for ss in wn.synsets(word[0], get_pos(word))]
                for sense in senses:
                    input_senses.append(sense)
            except:
                #lesk_result = wsd.lesk(input_sent, word[0])
                #input_lesk_senses[word[0]] = (lesk_result, lesk_result.definition())
                senses = [ss for ss in wn.synsets(word[0])]
                for sense in senses:
                    input_senses.append(sense)
        elif word[0] == my_word:
            my_pos = get_pos(word)
            lesk_result = wsd.lesk(input_sent, word[0])
            naive_lesk_result = (lesk_result, lesk_result.definition())
            
            

        else:
            continue
            
    return input_senses, naive_lesk_result

# parsed_input, my_pos, naive_lesk_result = parse_input(my_word, input_sent)
# print(parsed_input, my_pos)
# print('Naive lesk for ' + my_word + ': ', naive_lesk_result)


In [7]:
def get_hyper(word):
    pattern = '[a-z-]+'
    hyper = {}
    for ss in wn.synsets(word):
        try:
            hyper[ss.hypernyms()[0]] = ss.hypernyms()[0].definition()
        except IndexError:
            if ss.pos() == 's':
                lemma = ss.lemmas()[0]
                cs_head = re.findall(pattern, lemma.key())[-1]
                head_sets = wn.synsets(cs_head)[0]
                if head_sets.hypernyms():
                    hyper[head_sets.hypernyms()[0]] = head_sets.hypernyms()[0].definition()
                else:
                    hyper[head_sets] = head_sets.definition()


    
    return hyper

In [51]:
def main(my_word, input_sent, verbose=False): 
    
    parsed_input, naive_lesk_result = parse_input(my_word, input_sent)
    pattern = '[a-z-]+'
    count = 0
    sent_hypernyms = make_sent_hypernyms(input_sent)
    #print(sent_hypernyms)
    #best overall matches for each score that isn't matched to itself
    overall_best_pathsim_value = float('-inf')
    overall_best_pathsim_pair = ()
    overall_best_wup_value = float('-inf')
    overall_best_wup_pair = ()
    overall_best_lch_value = float('-inf')
    overall_best_lch_pair = ()
    overall_best_res_value = float('-inf')
    overall_best_res_pair = ()
    overall_best_jcn_value = float('-inf')
    overall_best_jcn_pair = ()
    overall_best_lin_value = float('-inf')
    overall_best_lin_pair = ()
    overall_best_lesk_value = 0
    overall_best_lesk_pair = ()

    #using all provided wordnet taxonomy methods
    #calculate similarity between hypernym sense of each input_sentence word sense
    #and hypernym sense of each word in the definitions of each concept sense's hypernym

    #synsets = wn.synsets(my_word) #list of senses for ambigous concept, without matching part of speech

    concept_data = dict.fromkeys(get_hyper(my_word).keys(), {})          
                                #will contain all similarity values between concept-sense-definition-word-senses
                                #and input sentence lesk-provided senses - d0
                                #who said quadruple-nested dictionaries were bad? I know I didn't!
    
    concept_sense_scores = dict.fromkeys(concept_data.keys(), {}) #maps concept sense hypernyms to its overall_best scores for each metric
    
    #print(concept_data, concept_sense_scores)
    #for each potential ambiguous concept hypernym sense - loop 0 (PCSH)
    for syn in concept_data:
        
        
        best_syn_scores = {'pathsim' : ((), float('-inf')),
                           'wup' : ((), float('-inf')),
                           'lch' : ((), float('-inf')),
                           'res' : ((), float('-inf')),
                           'jcn' : ((), float('-inf')),
                           'lin' : ((), float('-inf')),
                           'lesk' : ((), 0)
                          }
        
        definition = wt(syn.definition())
        def_pos = pos_tag(definition) #get pos_tags for current sense definition
        def_filtered = filter(definition) #remove stop words
        def_synsets = []
        
        syn_def = {}    #dictionary of definition-word-senses for current ambiguous concept sense
                        #maps to dictionary of definition-word-senses for each word of current sense's definition - d1
        
        #print(def_pos)
        #removes stopwords from PCSH definition and gets definition word senses
        for def_word in def_pos:
            if def_word[0] in def_filtered:
                #create list of current definition-word's senses
                current_pos = get_pos(def_word)
                def_synsets = wn.synsets(def_word[0], current_pos)
                
                if not def_synsets:
                    def_synsets = wn.synsets(def_word[0])
            
            hyp_synsets = []
            #print(def_synsets)   
            for ss in def_synsets:
                if ss.hypernyms() and ss.hypernyms()[0] not in hyp_synsets:
                    hyp_synsets.append(ss.hypernyms()[0])
                    inner = {ss.hypernyms()[0] : {}}
                    syn_def[ss] = inner


        #print(syn_def)
                        
     
        #for each nonstopword hypernym in the defintion of the current ambiguous concept sense
        #matches for part of speech - loop 1 (PCDSWH)
        for item in syn_def:
            syn_def_syn = sent_hypernyms
                            #dictionary of current concept-sense's definition's current word's hypernym
                            #mapped to dict of each input sentence word-sense's similarity score - d2
            
            item_head = list(syn_def[item].keys())[0]
            #print(item, item_head, syn_def_syn, '\n')

#             changed_item = False
#             item_head = ()
#             #handle wordnet's annoying satellite adjective tag
#             if item.pos() == 's':

#                 lemma = item.lemmas()[0]
#                 head = re.findall(pattern, lemma.key())[-1]
#                 head_sets = wn.synsets(head)

#                 #todo: figure out better way to handle multiple head adjective option

#                 item_head = [s for s in head_sets if s.pos() == 'a'][0]



            #calculate similarity between current definition-word sense and each input_sentence lesk-sense
            for input_word in syn_def_syn:
                best_current_input_similarity = {'pathsim_value' : None, #similarity dict for current input_sentence word - d3
                                                 'wup_value' : None,
                                                 'lch_value': None,
                                                 'res_value' : None,
                                                 'jcn_value' : None,
                                                 'lin_value' : None,
                                                 'lesk_value' : None 
                                                }
                for current_sense in syn_def_syn[input_word]:
                    same = False
                    similarity_scores = {'pathsim_value' : None, #similarity dict for current input_sentence word sense
                                         'wup_value' : None,
                                         'lch_value': None,
                                         'res_value' : None,
                                         'jcn_value' : None,
                                         'lin_value' : None,
                                         'lesk_value' : None 
                                        }

                    print(item_head, current_sense, syn_def_syn[input_word])
                    #current_sense = syn_def_syn[sense].key()

                    #lesk algorithm, modified from https://www.nltk.org/_modules/nltk/wsd.html
                    context = set(filter(wt(current_sense.definition())))

                    overlap = context.intersection(filter(wt(item_head.definition())))
                    similarity_scores['lesk_value'] = len(overlap)

                    if len(overlap) > best_syn_scores['lesk'][1]:
                        best_syn_scores['lesk'] = ((item_head, current_sense), len(overlap))







                    if current_sense.pos() == 's' and item_head.pos() == 'a':

                        lemma = current_sense.lemmas()[0]
                        cs_head = re.findall(pattern, lemma.key())[-1]
                        head_sets = wn.synsets(cs_head)
                        try:
                            current_sense = [s for s in head_sets if s.pos() == 'a'][0]

                            #print(current_sense, current_sense.definition(), definition)
                            if current_sense == item_head:
                                print('same adj: ', definition(), current_sense, current_sense.definition())
                        except:
                            continue
                    
                    elif current_sense.pos() != item_head.pos():
                        continue



                    if current_sense == item_head:
                        same = True

                    if len(overlap) > overall_best_lesk_value and not same:
                        overall_best_lesk_value = len(overlap)
                        overall_best_lesk_pair = (item_head, current_sense)


                    try:
                        #https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.path_similarity
                        pathsim_value = item_head.path_similarity(current_sense)
                        similarity_scores['pathsim_value'] = pathsim_value
                        if pathsim_value and pathsim_value > best_syn_scores['pathsim'][1]:
                            best_syn_scores['pathsim'] = ((item_head, current_sense), pathsim_value)

                        if pathsim_value and pathsim_value > overall_best_pathsim_value and not same:
                            overall_best_pathsim_value = pathsim_value
                            overall_best_pathsim_pair = (item_head, current_sense)

                    except Exception as e:
                        if type(e) != TypeError:
                            print(e)
                            continue


                    try:
                        #https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.wup_similarity
                        wup_value = item_head.wup_similarity(current_sense)
                        similarity_scores['wup_value'] = wup_value
                        if wup_value and wup_value > best_syn_scores['wup'][1]:
                            best_syn_scores['wup'] = ((item_head, current_sense), wup_value)

                        if wup_value and wup_value > overall_best_wup_value and not same:
                            overall_best_wup_value = wup_value
                            overall_best_wup_pair = (item_head, current_sense)
                            if wup_value >= 0.8:
                                print(item_head, current_sense, ' wup: ', wup_value)

                    except Exception as e:
                        print(item, sense, ' failed wup', e)

                    try:
                        #https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.lch_similarity
                        lch_value = item_head.lch_similarity(current_sense)
                        similarity_scores['lch_value'] = lch_value
                        if lch_value and lch_value > best_syn_scores['lch'][1]:
                            best_syn_scores['lch'] = ((item_head, current_sense), lch_value)

                        if lch_value and lch_value > overall_best_lch_value and not same:
                            overall_best_lch_value = lch_value
                            overall_best_lch_pair = (item_head, current_sense)
                            if lch_value >= 1.5:
                                print(item_head, current_sense, ' lch: ', lch_value)

                    except Exception as e:
                        print(item_head, current_sense, ' failed  lch', e)

                    try:
                        #https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.res_similarity
                        res_value = item_head.res_similarity(current_sense, ic)
                        similarity_scores['res_value'] = res_value
                        if res_value and res_value > best_syn_scores['res'][1]:
                            best_syn_scores['res'] = ((item_head, current_sense), res_value)

                        if res_value and res_value > overall_best_res_value and not same:
                            overall_best_res_value = res_value
                            overall_best_res_pair = (item_head, current_sense)
                            if res_value >= 2.5:  
                                print(item_head, current_sense, ' res: ', res_value)

                    except Exception as e:
                        print(item_head, current_sense, ' failed res', e)

                    try:
                        ##https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.jcn_similarity
                        jcn_value = item_head.jcn_similarity(current_sense, ic)
                        similarity_scores['jcn_value'] = jcn_value
                        if jcn_value and jcn_value > best_syn_scores['jcn'][1]:
                            best_syn_scores['jcn'] = ((item_head, current_sense), jcn_value)

                        if jcn_value and jcn_value > overall_best_jcn_value and not same:
                            overall_best_jcn_value = jcn_value
                            overall_best_jcn_pair = (item_head, current_sense)
                            if jcn_value >= 0.1:
                                print(item_head, current_sense, ' jcn: ', jcn_value)

                    except Exception as e:
                        print(item_head, current_sense, ' failed jcn', e)

                    try:
                        #https://www.nltk.org/api/nltk.corpus.reader.html#nltk.corpus.reader.wordnet.Synset.lin_similarity
                        lin_value = item_head.lin_similarity(current_sense, ic)
                        similarity_scores['lin_value'] = lin_value
                        if lin_value and lin_value > best_syn_scores['lin'][1]:
                            best_syn_scores['lin'] = ((item_head, current_sense), lin_value)

                        if lin_value and lin_value > overall_best_lin_value and not same:
                            overall_best_lin_value = lin_value
                            overall_best_lin_pair = (item_head, current_sense)
                            if lin_value >= 0.25:
                                print(item_head, current_sense, ' lin: ', lin_value)

                    except Exception as e:
                        print(item_head, current_sense, ' failed lin', e)


                    count += 1

            syn_dict[item].update(value_dict)

        concept_data[syn].update(syn_dict)

    concept_sense_scores[syn].update(best_syn_scores)


    print('overall_best_pathsim: ', overall_best_pathsim_pair, overall_best_pathsim_value)
    print('overall_best_wup: ', overall_best_wup_pair, overall_best_wup_value)
    print('overall_best_lch: ', overall_best_lch_pair, overall_best_lch_value)
    print('overall_best_res: ', overall_best_res_pair, overall_best_res_value)
    print('overall_best_jcn: ', overall_best_jcn_pair, overall_best_jcn_value)
    print('overall_best_lin: ', overall_best_lin_pair, overall_best_lin_value)
    print('overall_best_lesk: ', overall_best_lesk_pair, overall_best_lesk_value)
    print('naive_lesk', naive_lesk_result)

    print(count)
    
    if verbose:
        print(concept)
        
    #return concept, parsed_input

In [52]:
my_word = 'garden'
input_sent = 'I want to plant this rose in my new garden'
#print(parse_input(my_word, input_sent))
main(my_word, input_sent, verbose=False)
#print(parsed_input)
#input_sent_hypers = make_sent_hypernyms(input_sent)


Synset('region.n.03') Synset('condition.n.01') [Synset('condition.n.01'), Synset('state.n.02'), Synset('physical_entity.n.01'), Synset('desire.v.01'), Synset('be.v.01'), Synset('search.v.01'), Synset('communicate.v.01'), Synset('miss.v.06')]
Synset('region.n.03') Synset('condition.n.01')  jcn:  0.13960197487606174
Synset('region.n.03') Synset('state.n.02') [Synset('condition.n.01'), Synset('state.n.02'), Synset('physical_entity.n.01'), Synset('desire.v.01'), Synset('be.v.01'), Synset('search.v.01'), Synset('communicate.v.01'), Synset('miss.v.06')]
Synset('region.n.03') Synset('state.n.02')  lch:  1.55814461804655
Synset('region.n.03') Synset('state.n.02')  jcn:  0.16412322724351902
Synset('region.n.03') Synset('physical_entity.n.01') [Synset('condition.n.01'), Synset('state.n.02'), Synset('physical_entity.n.01'), Synset('desire.v.01'), Synset('be.v.01'), Synset('search.v.01'), Synset('communicate.v.01'), Synset('miss.v.06')]
Synset('region.n.03') Synset('physical_entity.n.01')  lch:  2

NameError: name 'syn_dict' is not defined